In [14]:
import pandas as pd
from tqdm import tqdm
from time import sleep
from password import passwordTrier
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
options = webdriver.ChromeOptions()
options.add_argument("--headless")
login_trier, password_trier = passwordTrier()

In [15]:
password_trier

'0788'

In [16]:
data = pd.read_excel('planilhas/genericos_filtrado.xls')

In [17]:
data

,codshopfarma,laboratorio,ean,produto,preco_venda,preco_compra,margem,preco_novo,margem_nova,erro
0,16982,ACHE/BIOSINTETICA,7.896182e+12,ACEBROFILINA XPE AD 120ML,36.50,12.57,190.373906,24.99,98.806683,NaN
1,113267,CIMED,7.896523e+12,ACEBROFILINA XPE AD 120ML,35.00,12.99,169.438029,24.99,92.378753,NaN
2,8731,NEO QUIMICA,7.896714e+12,ACEBROFILINA XPE AD 120ML,32.50,7.99,306.758448,24.99,212.765957,NaN
3,110884,TEUTO,7.896112e+12,ACEBROFILINA XPE AD 120ML,28.00,10.02,88.622754,24.99,149.401198,NaN
4,16545,NOVA QUIMICA,7.895296e+12,ACEBROFILINA XPE AD 120ML,37.05,0.00,inf,24.99,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
2235,16190,SANDOZ,7.897596e+12,ZOLPIDEM 10MG 30CP REV,82.00,8.48,866.981132,39.99,371.580189,NaN
2236,16923,EMS,7.896005e+12,ZOLPIDEM 10MG 30CP REV,72.00,6.74,968.249258,39.99,493.323442,NaN
2237,56518,NOVARTIS,7.897596e+12,ZOLPIDEM 10MG 30CP REV,49.90,6.99,613.876967,39.99,472.103004,NaN
2238,134104,GERMED,7.896005e+12,ZOLPIDEM 5MG 30CP SUBL,77.00,26.07,195.358650,69.99,168.469505,NaN


In [18]:
def inclui_preco_promocao(df, nome_promocao, data_inicial, data_final ,nova_promocao=True, filial=None):
    driver = webdriver.Chrome()#options=options)
    driver.get(f"http://179.187.94.169:4647/sgfpod1/Cad_0033.pod")
    sleep(1)
    driver.find_element(By.NAME, 'id_cod_usuario').send_keys(login_trier)
    sleep(1)
    driver.find_element(By.NAME, 'nom_senha').send_keys(password_trier)
    sleep(1)
    driver.find_element(By.NAME, 'login').click()
    sleep(1)
    if nova_promocao:
        if filial!=None:
            driver.find_element(By.NAME, 'cod_filialEntrada').send_keys(filial)
            driver.find_element(By.NAME, 'cod_filialEntrada').send_keys(Keys.RETURN)
            sleep(1)
        driver.find_element(By.NAME, 'nom_desconto').send_keys(nome_promocao)
        driver.find_element(By.NAME, 'nom_desconto').send_keys(Keys.RETURN)
        sleep(1)
        driver.find_element(By.NAME, 'dat_inicio').send_keys(data_inicial)
        driver.find_element(By.NAME, 'dat_inicio').send_keys(Keys.RETURN)
        sleep(1)
        driver.find_element(By.NAME, 'dat_fim').send_keys(data_final)
        driver.find_element(By.NAME, 'dat_fim').send_keys(Keys.RETURN)
        sleep(1)
    for index, row in tqdm(df.iterrows(), total=df.shape[0]):
        
        #driver.find_element(By.NAME, 'cadprodu_reduzido_cod_barraEntrada').send_keys(row['codshopfarma'])
        #driver.find_element(By.NAME, 'cadprodu_reduzido_cod_barraEntrada').send_keys(Keys.RETURN)
        sleep(10)
        erro_duplicacao = (driver.find_element(By.NAME, 'sel_produs')).text # verifica se já tem um produto na promoção
        print(erro_duplicacao)
        if erro_duplicacao != None:
            print('Um produto já está na promoção')
        else:
            print('Não tem produto na promoção')
        sleep(10)
        preco_novo = str(row['preco_novo']).replace('.',',')
        driver.find_element(By.NAME, 'vlr_promocao').send_keys(preco_novo)
        sleep(1)
        driver.find_element(By.NAME, 'salvar').click()
        sleep(1)
    print(row['codshopfarma'],row['produto'], row['preco_novo'])

In [61]:
def cadastro_depto(df, depto):
    driver = webdriver.Chrome()#options=options)
    driver.get(f"http://179.187.94.169:4647/sgfpod1/Cad_0058.pod")
    sleep(1)
    driver.find_element(By.NAME, 'id_cod_usuario').send_keys(login_trier)
    sleep(1)
    driver.find_element(By.NAME, 'nom_senha').send_keys(password_trier)
    sleep(1)
    driver.find_element(By.NAME, 'login').click()
    sleep(1)
    for index, row in tqdm(df.iterrows(), total=df.shape[0]):
        sleep(1)
        driver.find_element(By.NAME, 'id_cod_reduzidoEntrada').send_keys(row['codshopfarma'])
        driver.find_element(By.NAME, 'id_cod_reduzidoEntrada').send_keys(Keys.RETURN)
        sleep(1)
        driver.find_element(By.NAME, 'cod_deptoEntrada').send_keys(depto)
        driver.find_element(By.NAME, 'cod_deptoEntrada').send_keys(Keys.RETURN)
        sleep(1)
        driver.find_element(By.NAME, 'salvar').click()

In [62]:
cadastro_depto(data, '11')

100%|██████████| 2240/2240 [1:57:46<00:00,  3.15s/it]


In [19]:
inclui_preco_promocao(data, 'genéricos a preços imbativeis', '23/03/2023', '30/04/2023', filial='1')

  0%|          | 0/2240 [00:00<?, ?it/s]

 
Um produto já está na promoção


  0%|          | 1/2240 [00:22<13:47:46, 22.18s/it]

 
Um produto já está na promoção


  0%|          | 1/2240 [00:43<26:53:13, 43.23s/it]


KeyboardInterrupt: 